# Automate Event Planning


Requirements:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Crew, Task

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

# these expect to find a .env file at the directory above the lesson.                                                                                                                     # the format for that file is (without the comment)                                                                                                                                       #API_KEYNAME=AStringThatIsTheLongAPIKeyFromSomeService
def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    return openai_api_key

def get_serper_api_key():
    load_env()
    openai_api_key = os.getenv("SERPER_API_KEY")
    return openai_api_key


# break line every 80 characters if line is longer than 80 characters
# don't break in the middle of a word
def pretty_print_result(result):
  parsed_result = []
  for line in result.split('\n'):
      if len(line) > 80:
          words = line.split(' ')
          new_line = ''
          for word in words:
              if len(new_line) + len(word) + 1 > 80:
                  parsed_result.append(new_line)
                  new_line = word
              else:
                  if new_line == '':
                      new_line = word
                  else:
                      new_line += ' ' + word
          parsed_result.append(new_line)
      else:
          parsed_result.append(line)
  return "\n".join(parsed_result)


In [4]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [6]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [7]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [8]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [9]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [10]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [11]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [12]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since by setting `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [13]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [15]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in San Francisco that meets the criteria for the Tech Innovation Conference. I must consider the theme, size, and budget constraints of the event.

Action: Search the internet
Action Input: {"search_query": "tech conference venues in San Francisco"} 


Search results: Title: Discover Tech Conferences Events & Activities in San Francisco, CA
Link: https://www.eventbrite.com/d/ca--san-francisco/tech-conferences/
Snippet: Tech conferences events in San Francisco, CA ; Time for a Tech Reformation? With Greg Epstein. Mon, Nov 18 • 5:30 PM. The Commonwealth Club · The Commonwealth ...
---
Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: With over 200 speakers, the San 

This is the agent final answer: After researching and reading the website content, I have found the following potential conference venues in San Francisco that may accommodate the Tech Innovation Conference:
1. The American Bookbinders Museum - Located in South of Market (SOMA), San Francisco, CA. The venue has a capacity of 110 guests and offers a turn-key studio & meeting space. The hire fee is $375 per hour.
2. The Producer's Loft Studio - Located in Mission District, San Francisco, CA. The venue can accommodate 70-100 guests and offers an event area for hire at $250 per hour.
3. Dynamico Space - Located in Union Square, San Francisco, CA. The venue can accommodate 50 guests and offers a hire fee of $250 per hour.
4. Dogpatch Studios - Located in San Francisco, CA. The venue can accommodate 110-140 guests and offers space for hire at $3000 per session.
5. AMA's SF Executive Conference Center - Located in South of Market (SOMA), San Francisco, CA. The venue can accommodate 138 guests

Thought: I need to find information specifically about the Tech Innovation Conference to effectively market it and engage with potential attendees.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference San Francisco details"} 


Search results: Title: Event Rental Space FAQ's | San Francisco
Link: https://bookbindersmuseum.org/rental/rental-faqs/
Snippet: What is included with my rental? Is music ok? Catering kitchen? Learn much more about our San Francisco event rental spaces here.
---
Title: American Bookbinders Museum | Best SF Caterer Event Venues
Link: https://www.ggcatering.com/venues/the-american-bookbinders-museum
Snippet: All of the binding equipment is fully operational just as it once was in the 19th century and can be operated by docents during events.
---
Title: American Bookbinders Museum | San Francisco
Link: https://bookbindersmuseum.org/
Snippet: The American Bookbinders Museum is the only museum of its kind in North America. Come, see

Thought: I need to gather more information about the Tech Innovation Conference in San Francisco to effectively market it and engage with potential attendees.

Action: Read website content
Action Input: {"website_url": "https://techsummit.tech/san-francisco-tech-conference/?srsltid=AfmBOooochaej3CW8j6x4g4Qcmpk9kDATmN9yhSVtINAsqiWHDG_r5Kw"} 

Event Details San Francisco – Tech Summit
Tech Summit
Events
Tech Summit Silicon Valley
Event Details San Francisco Speakers Silicon Valley Agenda Venue, Travel & Hotels Get Tickets 
Tech Summit London
Agenda London Event Details Speakers London Get Tickets 
Tech Summit Europe
Event Details Speakers Agenda Get Tickets 
Tech Summit Asia
Speakers Asia Get Tickets 
Texas Conference
Event Details Texas Speakers Texas 
Partnerships
Sponsor Event & Media Partners Intelligence
Event Information
Convince Your Boss Tech Startups Tech Investors Tech Awards International Attendees FAQ’s Contact Us Volunteers Social Responsibility
Discover
						Tech Summit 
H

Exception in thread Thread-5 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/us

Thought: I need to gather more information about the Tech Innovation Conference in San Francisco to effectively market it and engage with potential attendees.

Action: Read website content
Action Input: {"website_url": "https://techsummit.tech/san-francisco-tech-conference/?srsltid=AfmBOooochaej3CW8j6x4g4Qcmpk9kDATmN9yhSVtINAsqiWHDG_r5Kw"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to gather more information about the Tech Innovation Conference in San Francisco to effectively market it and engage with potential attendees.

Action: Read website content
Action Input: {"website_url": "https://techsummit.tech/san-francisco-tech-conference/?srsltid=AfmBOorXydonRpWJ8sMkxKDRPVLf_JJFT-68yBY897hqAkQPAwI0WRC8"} 

Event Details San Francisco – Tech Summit
Tech Summit
Events
Tech Summit Silicon Valley
Event Details San Francisco Speakers Silicon Valley Agenda Venue, Travel & Hotels Get Tickets 
Tech Summit London
Agend

In [16]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'South of Market (SOMA), San Francisco, CA',
 'booking_status': '$375 per hour',
 'capacity': 110,
 'name': 'The American Bookbinders Museum'}


In [18]:
from IPython.display import Markdown
Markdown("marketing_report.md")

# Marketing Activities Report

## Tech Innovation Conference Promotion

### Event Details
- **Name:** Tech Innovation Conference
- **Location:** San Francisco, CA
- **Dates:** TBD
- **Speakers:** 150+
- **Duration:** 2 Days
- **Ticket Prices:**
  - Standard Ticket: $795 ($695 early bird)
  - Business Ticket: $1495 ($795 early bird)
  - Executive Ticket: $3495 ($2995 early bird)

### Highlights
- 150+ Speakers - Experts and Leaders in Tech
- Masterclasses, Workshops, Roundtables
- Professional Development
- Networking and Opportunities

### Why Attend
1. Tech leaders, innovators, thought leaders, and more
2. Two days of unmissable conference talks
3. Multiple content tracks covering technical and non-technical content
4. Thousands of attendees local, national, and beyond
5. Vast range of opportunities, offers, exclusive perks from partners, and more
6. Unique learning and development opportunities at roundtables, workshops, and masterclasses
7. Exclusive inside track from world leaders in tech
8. Global partners including well-loved brands
9. Thousands of emerging companies from across the world making their mark
10. Silicon Valley as the bustling epicenter of Tech and our iconic venue
11. A one-of-a-kind networking experience
12. Q&As and experiences - interact with speakers and ask them anything
13. Excellent value for money with bigger and better offerings in 2024

### Call to Action
- Register interest
- Get tickets
- Partner with us
- Apply to speak
- Contact us

### Additional Information
- **Event Website:** [Tech Summit San Francisco](https://techsummit.tech/san-francisco-tech-conference/?srsltid=AfmBOorXydonRpWJ8sMkxKDRPVLf_JJFT-68yBY897hqAkQPAwI0WRC8)
- **Copyright:** © 2024 Tech Summit. All rights reserved

---
Thank you for giving me the opportunity to research and provide a comprehensive report on promoting the Tech Innovation Conference in San Francisco.